In [140]:
import pandas as pd
import random
import numpy as np
import math
seeds = pd.read_csv('seeds.csv')

In [141]:
class ProcessingData:
    
    @staticmethod
    def shuffleData(base):
        for x in range(len(base)-1,0,-1):
            newPlace = random.randint(0,x-1)
            base.loc[x],base.loc[newPlace]=base.loc[newPlace],base.loc[x]
            
    @staticmethod
    def normalizeData(base):
        base.iloc[:,:-1] = (base.iloc[:,:-1]-base.iloc[:,:-1].min())/(base.iloc[:,:-1].max()-base.iloc[:,:-1].min())

    @staticmethod
    def splitData(base,x,y):
        n = int(len(base)*x/100)
        base_tren = base[:n]
        base_valid = base[n:]
        return(base_tren,base_valid)

In [142]:
seedsEdit = seeds.copy()

ProcessingData.shuffleData(seedsEdit)
ProcessingData.normalizeData(seedsEdit)
trening_data,validate_data = ProcessingData.splitData(seedsEdit,70,30)

In [143]:
class SoftSet:
    
    @staticmethod
    def recoverTreningData(data):
        sortedData = []
        sort = data.groupby(data.Type)
        sortedData.append(sort.get_group(1))
        sortedData.append(sort.get_group(2))
        sortedData.append(sort.get_group(3))

        return sortedData

    @staticmethod
    def meanMaker(sortedData):
        meanTable = []
        for j in range(len(sortedData)):
            meanTable.append(sortedData[j].loc[:,sortedData[j].columns != "Type"].mean(numeric_only=True).tolist())
        return meanTable

    @staticmethod
    def make10Table(data,meanTable):
        oneZeroTable = []
        for j in range(len(data)):
            speciesTable = []
            for k in range(len(data[j].columns)-1):
                smaller = data[j][data[j].iloc[:,k] < meanTable[j][k]].count()
                bigger = len(data[j]) - smaller
                
                if smaller[k] > bigger[k]:
                    speciesTable.extend([1,0])
                else:
                     speciesTable.extend([0,1])
            oneZeroTable.append(speciesTable)
        return oneZeroTable

    @staticmethod
    def softSet(validate_data,oneZeroTable,meanTable):
        correct = 0
        varieties = [1,2,3]
        for i in range(len(validate_data)):
            trueName = validate_data.iloc[i].Type
            oneZeroValidate = []
            for j in range(len(meanTable)):
                oneZeroValidateOne = []
                for k in range(len(validate_data.iloc[i])-1):
                    if validate_data.iloc[i][k] < meanTable[j][k]:
                        oneZeroValidateOne.extend([1,0])
                    else:
                        oneZeroValidateOne.extend([0,1])
                oneZeroValidate.append(oneZeroValidateOne)
                 
            tableOfSums = []
            for j in range(len(oneZeroValidate)):
                suma = 0
                for sum in range(len(oneZeroValidate[j])):
                    suma+=oneZeroValidate[j][sum]*oneZeroTable[j][sum]
                tableOfSums.append(suma)
                

            guessedName = varieties[np.argmax(tableOfSums)]

            if guessedName == trueName:
                correct+=1
        return correct,correct/len(validate_data)*100

In [144]:
sortedData = SoftSet.recoverTreningData(trening_data)
meanTable = SoftSet.meanMaker(sortedData)
oneZeroTable = SoftSet.make10Table(sortedData,meanTable)
correct, accuracy = SoftSet.softSet(validate_data,oneZeroTable,meanTable)
print("{}/{} accuracy => {:.2f}%".format(correct,len(validate_data),accuracy))

14/60 accuracy => 23.33%
